IMPORTING ALL NECESSARY MODULES


In [127]:
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef

LOADING IN THE DATASET


In [128]:
df=pd.read_csv("Threats (1).csv")

FINDING THE MISSING VALUES AND DEALING WITH THEM

In [129]:
df['service'] = df['service'].replace({'-':np.nan})
df['service'].unique() #'-' from the service column has been set to null(nan)

array([nan, 'dns', 'ssh', 'ftp-data', 'http', 'ftp', 'smtp', 'radius',
       'pop3', 'dhcp', 'ssl', 'snmp', 'irc'], dtype=object)

SPLITTING THE DATASET INTO DEPENDENT AND INDEPENDENT FEATURES


In [130]:
X=df.drop("attack_cat",axis=1).copy()
Y=df["attack_cat"].copy()

ONE HOT ENCODING FOR CHARACTER COLUMNS IN THE DEPENDENT PART OF THE DATASET(X)

In [131]:
X_encoded=pd.get_dummies(X,columns=['service','proto','state'],dtype=int)
print(X_encoded.isnull().sum().sum()) #no null values in dataset


0


LABEL ENCODING THE ATTACK_CAT COLUMN IN THE INDEPENDENT PART OF DATASET(Y)

In [132]:
from sklearn.preprocessing import LabelEncoder
YY=Y.tolist() #converting attack_cat column into a list for label encoding
target=YY
le=LabelEncoder()
target_encoded=le.fit_transform(target)
target_encoded
Y=pd.DataFrame(target_encoded,columns=['attack_cat'])
Y


,attack_cat
0,6
1,6
2,6
3,6
4,6
...,...
135336,5
135337,8
135338,5
135339,5


TRAIN_TEST SPLIT 

In [133]:
X_train,X_test,Y_train, Y_test=train_test_split(X_encoded,Y,random_state=42)


CREATING THE DECISION TREE

In [134]:
clf=DecisionTreeClassifier(ccp_alpha=0.0001, min_impurity_decrease=1e-06,
                       min_samples_leaf=2, random_state=42)
clf=clf.fit(X_train,Y_train)

GETTING THE PREDICTIONS ARRAY AND THE PREDICTION PROBABILITIES ARRAY MADE BY OUR CLASSIFICATION TREE TO DETERMINE PERFORMANCE METRICS

In [135]:
predictions=clf.predict(X_test)
predictionss=clf.predict_proba(X_test)

CALCULATING PERFORMANCE METRICS TO JUDGE THE MODEL

In [136]:
print(classification_report(Y_test,predictions,target_names=['Normal', 'Backdoor', 'Analysis', 'Fuzzers', 'Shellcode',
       'Reconnaissance', 'Exploits', 'DoS', 'Worms', 'Generic']))

                precision    recall  f1-score   support

        Normal       0.76      0.21      0.33       487
      Backdoor       0.72      0.19      0.30       442
      Analysis       0.43      0.16      0.24      3101
       Fuzzers       0.64      0.91      0.75      8357
     Shellcode       0.94      0.87      0.90      4504
Reconnaissance       1.00      0.98      0.99      9951
      Exploits       1.00      1.00      1.00      4047
           DoS       0.91      0.76      0.83      2618
         Worms       0.63      0.73      0.68       290
       Generic       0.66      0.64      0.65        39

      accuracy                           0.83     33836
     macro avg       0.77      0.65      0.67     33836
  weighted avg       0.83      0.83      0.82     33836



In [137]:
#ACCURACY
accuracy_score(Y_test,predictions)


0.8343775860030737

In [ ]:
#AUC 
YY_test=Y_test.to_numpy()
auc = roc_auc_score(YY_test,predictionss,multi_class='ovr')
print(f"AUC: {auc}")


AUC: 0.9575260530470097


In [ ]:
#F1 SCORE
f1 = f1_score(YY_test, predictions, average='weighted')
print(f"F1 Score (weighted): {f1}")

F1 Score (weighted): 0.8174392302315107


In [141]:
#MATTHEWS CORRELATION COEFFICIENT
mcc = matthews_corrcoef(YY_test, predictions)
print(f"Matthews Correlation Coefficient (MCC): {mcc}")

Matthews Correlation Coefficient (MCC): 0.7976427964883681


In [ ]:
#RECALL RATE
recall = recall_score(YY_test, predictions, average='weighted')  # 'micro' or 'weighted' are also options
print(f"Recall (macro-average): {recall}")

Recall (macro-average): 0.8343775860030737


FINAL PLOT OF DECISION TREE

In [138]:
plt.figure(figsize=(100, 100))
class_names = ['Normal', 'Backdoor', 'Analysis', 'Fuzzers', 'Shellcode',
       'Reconnaissance', 'Exploits', 'DoS', 'Worms', 'Generic']
# Plot the decision tree with the actual feature names
plot_tree(     clf, 
               filled=True,
               fontsize=8,
               class_names=class_names,
               feature_names=X_train.columns)  # Pass the feature names here

plt.show()

ALL PERFORMANCE METRICS INDICATE A DECENTLY WELL PREDICTING CLASSIFICATION TREE WITH AN ACCURACY OF ABOUT 83%